In [ ]:
# check CPU status
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Import the necessary Library and Files
from osgeo import gdal_array
from osgeo import gdal
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.metrics import r2_score

In [ ]:
# load the test dataset
filename="/content/drive/My Drive/GIS/test_dataset_canada.dat"
raster = gdal.Open(filename)
rasterArray = raster.ReadAsArray()
tb1 = rasterArray[0,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb2 = rasterArray[1,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb3 = rasterArray[2,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb4 = rasterArray[3,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb5 = rasterArray[4,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb6 = rasterArray[5,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb1_reshape=np.reshape(tb1,(tb1.shape[0]*tb1.shape[1],))
tb3_reshape=np.reshape(tb3,(tb3.shape[0]*tb3.shape[1],))
tb4_reshape=np.reshape(tb4,(tb4.shape[0]*tb4.shape[1],))
tb5_reshape=np.reshape(tb5,(tb5.shape[0]*tb5.shape[1],))
tb6_reshape=np.reshape(tb6,(tb6.shape[0]*tb6.shape[1]))
tb2_reshape=np.reshape(tb2,(tb2.shape[0]*tb2.shape[1],))

In [ ]:
tile_size=3
test_x=np.stack((tb1_reshape,tb3_reshape,tb5_reshape),axis=-1)
test_y=tb2_reshape
list_small_tiles=[]
list_small_tiles.clear()
for i in range(test_x.shape[0]-tile_size):
  list_small_tiles.append(test_x[i:i+tile_size])
list_small_tiles.append(test_x[i:i+tile_size])
list_small_tiles.append(test_x[i+1:i+tile_size+1])

In [ ]:
test_x=np.array(list_small_tiles)
test_x=np.reshape(test_x,(test_x.shape[0],tile_size,tile_size,1))
test_y=test_y[:-tile_size+2]

In [ ]:
#check for x and y shape
print(test_x.shape)
print(test_y.shape)

In [ ]:
# load pre trained model
from keras.models import load_model
model=load_model('../trained_models/CNN-Train-four-band.h5')

In [ ]:
#whole test data performance evaluations
pred_y=model.predict(test_x)
print("r2 score: %.6f" %r2_score(test_y,pred_y))
rmse = sqrt(mean_squared_error(test_y,pred_y))
print('Test RMSE: %.3f' % rmse)
# calculate MAE
mae = mean_absolute_error(test_y,pred_y)
print('Test MAE: %.3f' % mae)

In [ ]:
# prepare individual cases
j=0
case1_x=[]
case1_y=[]
case2_x=[]
case2_y=[]
case3_x=[]
case3_y=[]
case4_x=[]
case4_y=[]
case5_x=[]
case5_y=[]
case6_x=[]
case6_y=[]
case7_x=[]
case7_y=[]
case8_x=[]
case8_y=[]
case9_x=[]
case9_y=[]
for i in range(tb1.size):
    if(tb1_reshape[i]<tb2_reshape[i]<tb3_reshape[i]):
        case1_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case1_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]<tb2_reshape[i]==tb3_reshape[i]):
        case2_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case2_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]<tb2_reshape[i]>tb3_reshape[i]):
        case3_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case3_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]==tb2_reshape[i]<tb3_reshape[i]):
        case4_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case4_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]==tb2_reshape[i]==tb3_reshape[i]):
        case5_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case5_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]==tb2_reshape[i]>tb3_reshape[i]):
        case6_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case6_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]>tb2_reshape[i]<tb3_reshape[i]):
        case7_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case7_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]>tb2_reshape[i]==tb3_reshape[i]):
        case8_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case8_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]>tb2_reshape[i]>tb3_reshape[i]):
        case9_x.append([tb1_reshape[i],tb3_reshape[i],tb5_reshape[i]])
        case9_y.append(tb2_reshape[i])

In [ ]:
def prepare_case(x,y):
    x=np.array(x)
    y=np.array(y)
    small_tiles=[]
    small_tiles.clear()
    for i in range(x.shape[0]-tile_size):
        small_tiles.append(x[i:i+tile_size])
    small_tiles.append(x[i:i+tile_size])
    small_tiles.append(x[i+1:i+tile_size+1])
    test_x=np.array(small_tiles)\
    test_x=np.reshape(test_x,(test_x.shape[0],tile_size,tile_size,1))
    test_y=y[:-tile_size+2]
    return (test_x,test_y)

In [ ]:
def predict_case(test_x,test_y):
    pred_y=model.predict(test_x)
    print("r2 score: %.6f" %r2_score(test_y,pred_y))
    rmse = sqrt(mean_squared_error(test_y,pred_y))
    print('Test RMSE: %.3f' % rmse)
    # calculate MAE
    mae = mean_absolute_error(test_y,pred_y)
    print('Test MAE: %.3f' % mae)
    return pred_y

In [ ]:
# predict output for indivdual cases
x1,y1=prepare_case(case1_x,case1_y)
y1_hat=predict_case(x1,y1)
x3,y3=prepare_case(case3_x,case3_y)
y3_hat=predict_case(x3,y3)
x7,y7=prepare_case(case7_x,case7_y)
y7_hat=predict_case(x7,y7)
x9,y9=prepare_case(case9_x,case9_y)
y9_hat=predict_case(x9,y9)